# 14 - CNN for image classification en Keras

by [Fabio A. González](http://dis.unal.edu.co/~fgonza/), Universidad Nacional de Colombia

version 1.0, June 2018

## Part of the class [Applied Deep Learning](https://github.com/albahnsen/AppliedDeepLearningClass)


This notebook is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). 

This tutorial is based on the Blog post from Francois Chollet []"Building powerful image classification models using very little data"](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

## Download data

This is a dataset of images of cats and dogs that was extacted from this [Kaggle competition](https://www.kaggle.com/c/dogs-vs-cats/data). 

The file can be manually download from [here](https://drive.google.com/file/d/1Ys08yalsbM29fvvPhyDxv-SV7TBg3VxK/view?usp=sharing)

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1Ys08yalsbM29fvvPhyDxv-SV7TBg3VxK'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])

After downloading the file, we uncompress it*

In [0]:
!unzip cnn_data.zip > /dev/null


This creates the folder data:

In [0]:
!ls -la


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

img = load_img('data/train/dog/dog.125.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
plt.imshow(img)

## Data augmentation

In [0]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/train/cat/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, _, _)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, _, _)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely
plt.imshow(batch[0]/255) 

## Conv Net for Dog-Cat discrimination

Training variables

In [0]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 400
epochs = 50
batch_size = 16

The convolutional network model

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [0]:
epochs = 10
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Testing the classifier

We will use the validation generator to generate one batch of samples and we'll pass them to the model

In [0]:
for x_batch, y_batch in validation_generator:
  plt.imshow(x_batch[0])
  print(y_batch)
  print(model.predict_proba(x_batch))
  break

## Transfer Learning

We will use a pretrained image classification model from Keras. They are documented [here](https://keras.io/applications/)

In [18]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 128, 128

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 400
epochs = 50
batch_size = 16

datagen = ImageDataGenerator(rescale=1. / 255)

# build the MobileNet network
model = applications.MobileNet(include_top=False, weights='imagenet'
                               , input_shape=(img_width, img_height, 3))

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, nb_train_samples // batch_size)
np.save('bottleneck_features_train.npy',
        bottleneck_features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, nb_validation_samples // batch_size)
np.save('bottleneck_features_validation.npy',
        bottleneck_features_validation)

17235968/17225924 [==============================] - 0s 0us/step
Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [19]:
bottleneck_features_train.shape

(2000, 4, 4, 1024)

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 130, 130, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 66, 66, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
__________

In [21]:
train_data = np.load('bottleneck_features_train.npy')
train_labels = np.array(
    [0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

validation_data = np.load('bottleneck_features_validation.npy')
validation_labels = np.array(
    [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

model_1 = Sequential()
model_1.add(Flatten(input_shape=train_data.shape[1:]))
model_1.add(Dense(256, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

model_1.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model_1.save_weights(top_model_weights_path)

Train on 2000 samples, validate on 400 samples
Epoch 1/50
2000/2000 [==============================] - 2s 1ms/step - loss: 2.0776 - acc: 0.8405 - val_loss: 1.0161 - val_acc: 0.9150
Epoch 2/50
2000/2000 [==============================] - 2s 767us/step - loss: 0.5668 - acc: 0.9510 - val_loss: 1.2849 - val_acc: 0.9075
Epoch 3/50
2000/2000 [==============================] - 2s 755us/step - loss: 0.5890 - acc: 0.9530 - val_loss: 0.4222 - val_acc: 0.9600
Epoch 4/50
2000/2000 [==============================] - 2s 760us/step - loss: 0.4038 - acc: 0.9660 - val_loss: 0.3919 - val_acc: 0.9625
Epoch 5/50
2000/2000 [==============================] - 1s 740us/step - loss: 0.3896 - acc: 0.9680 - val_loss: 0.4921 - val_acc: 0.9500
Epoch 6/50
2000/2000 [==============================] - 1s 745us/step - loss: 0.3383 - acc: 0.9745 - val_loss: 0.3419 - val_acc: 0.9700
Epoch 7/50
1856/2000 [==========================>...] - ETA: 0s - loss: 0.2730 - acc: 0.9763

2000/2000 [==============================] - 1s 738us/step - loss: 0.2758 - acc: 0.9760 - val_loss: 0.3754 - val_acc: 0.9625
Epoch 8/50
2000/2000 [==============================] - 1s 735us/step - loss: 0.3422 - acc: 0.9710 - val_loss: 0.5928 - val_acc: 0.9550
Epoch 9/50
2000/2000 [==============================] - 1s 735us/step - loss: 0.3417 - acc: 0.9750 - val_loss: 0.4081 - val_acc: 0.9700
Epoch 10/50
2000/2000 [==============================] - 1s 732us/step - loss: 0.2052 - acc: 0.9840 - val_loss: 0.3124 - val_acc: 0.9750
Epoch 11/50
2000/2000 [==============================] - 1s 730us/step - loss: 0.2088 - acc: 0.9825 - val_loss: 0.4237 - val_acc: 0.9675
Epoch 12/50
2000/2000 [==============================] - 1s 732us/step - loss: 0.2859 - acc: 0.9770 - val_loss: 0.4679 - val_acc: 0.9625
Epoch 13/50
2000/2000 [==============================] - 1s 729us/step - loss: 0.1647 - acc: 0.9860 - val_loss: 0.4797 - val_acc: 0.9650
Epoch 14/50
 736/2000 [==========>...................] 

2000/2000 [==============================] - 1s 735us/step - loss: 0.1402 - acc: 0.9880 - val_loss: 0.3411 - val_acc: 0.9675
Epoch 15/50
2000/2000 [==============================] - 1s 736us/step - loss: 0.1289 - acc: 0.9885 - val_loss: 0.4714 - val_acc: 0.9550
Epoch 16/50
2000/2000 [==============================] - 1s 734us/step - loss: 0.1175 - acc: 0.9900 - val_loss: 0.4735 - val_acc: 0.9675
Epoch 17/50
2000/2000 [==============================] - 1s 738us/step - loss: 0.0940 - acc: 0.9900 - val_loss: 0.4334 - val_acc: 0.9650
Epoch 18/50
2000/2000 [==============================] - 1s 726us/step - loss: 0.1631 - acc: 0.9880 - val_loss: 0.4650 - val_acc: 0.9675
Epoch 19/50
2000/2000 [==============================] - 1s 729us/step - loss: 0.1190 - acc: 0.9905 - val_loss: 0.2557 - val_acc: 0.9725
Epoch 20/50
2000/2000 [==============================] - 1s 744us/step - loss: 0.0801 - acc: 0.9940 - val_loss: 0.4501 - val_acc: 0.9650
Epoch 21/50
 720/2000 [=========>....................

2000/2000 [==============================] - 1s 742us/step - loss: 0.0713 - acc: 0.9940 - val_loss: 0.4418 - val_acc: 0.9650
Epoch 22/50
2000/2000 [==============================] - 1s 740us/step - loss: 0.0731 - acc: 0.9935 - val_loss: 0.2773 - val_acc: 0.9800
Epoch 23/50
2000/2000 [==============================] - 1s 730us/step - loss: 0.0440 - acc: 0.9960 - val_loss: 0.4714 - val_acc: 0.9575
Epoch 24/50
2000/2000 [==============================] - 1s 731us/step - loss: 0.0552 - acc: 0.9960 - val_loss: 0.3682 - val_acc: 0.9750
Epoch 25/50
2000/2000 [==============================] - 1s 736us/step - loss: 0.0607 - acc: 0.9950 - val_loss: 0.4103 - val_acc: 0.9650
Epoch 26/50
2000/2000 [==============================] - 1s 736us/step - loss: 0.0426 - acc: 0.9960 - val_loss: 0.4356 - val_acc: 0.9675
Epoch 27/50
2000/2000 [==============================] - 1s 731us/step - loss: 0.0468 - acc: 0.9960 - val_loss: 0.3781 - val_acc: 0.9675
Epoch 28/50
 656/2000 [========>.....................

2000/2000 [==============================] - 1s 734us/step - loss: 0.0611 - acc: 0.9960 - val_loss: 0.2500 - val_acc: 0.9825
Epoch 29/50
2000/2000 [==============================] - 1s 722us/step - loss: 0.0501 - acc: 0.9965 - val_loss: 0.2739 - val_acc: 0.9775
Epoch 30/50
2000/2000 [==============================] - 1s 732us/step - loss: 0.0540 - acc: 0.9965 - val_loss: 0.2785 - val_acc: 0.9775
Epoch 31/50
2000/2000 [==============================] - 1s 736us/step - loss: 0.0454 - acc: 0.9970 - val_loss: 0.3812 - val_acc: 0.9700
Epoch 32/50
2000/2000 [==============================] - 1s 719us/step - loss: 0.0320 - acc: 0.9980 - val_loss: 0.3970 - val_acc: 0.9700
Epoch 33/50
2000/2000 [==============================] - 1s 728us/step - loss: 0.0400 - acc: 0.9970 - val_loss: 0.4154 - val_acc: 0.9700
Epoch 34/50
2000/2000 [==============================] - 1s 734us/step - loss: 0.0408 - acc: 0.9970 - val_loss: 0.2069 - val_acc: 0.9800
Epoch 35/50
 656/2000 [========>.....................

2000/2000 [==============================] - 1s 715us/step - loss: 0.0440 - acc: 0.9970 - val_loss: 0.3551 - val_acc: 0.9675
Epoch 36/50
2000/2000 [==============================] - 1s 726us/step - loss: 0.0420 - acc: 0.9970 - val_loss: 0.2969 - val_acc: 0.9750
Epoch 37/50
2000/2000 [==============================] - 1s 723us/step - loss: 0.0320 - acc: 0.9980 - val_loss: 0.3469 - val_acc: 0.9675
Epoch 38/50
2000/2000 [==============================] - 1s 711us/step - loss: 0.0336 - acc: 0.9975 - val_loss: 0.3271 - val_acc: 0.9675
Epoch 39/50
2000/2000 [==============================] - 1s 717us/step - loss: 0.0363 - acc: 0.9975 - val_loss: 0.3319 - val_acc: 0.9700
Epoch 40/50
2000/2000 [==============================] - 1s 722us/step - loss: 0.0378 - acc: 0.9975 - val_loss: 0.2960 - val_acc: 0.9750
Epoch 41/50
2000/2000 [==============================] - 1s 729us/step - loss: 0.0321 - acc: 0.9980 - val_loss: 0.3913 - val_acc: 0.9675
Epoch 42/50
 720/2000 [=========>....................

2000/2000 [==============================] - 1s 723us/step - loss: 0.0481 - acc: 0.9970 - val_loss: 0.2380 - val_acc: 0.9775
Epoch 43/50
2000/2000 [==============================] - 1s 720us/step - loss: 0.0320 - acc: 0.9980 - val_loss: 0.2390 - val_acc: 0.9775
Epoch 44/50
2000/2000 [==============================] - 1s 736us/step - loss: 0.0333 - acc: 0.9975 - val_loss: 0.2492 - val_acc: 0.9775
Epoch 45/50
2000/2000 [==============================] - 1s 727us/step - loss: 0.0348 - acc: 0.9975 - val_loss: 0.2313 - val_acc: 0.9800
Epoch 46/50
2000/2000 [==============================] - 1s 711us/step - loss: 0.0320 - acc: 0.9980 - val_loss: 0.2236 - val_acc: 0.9800
Epoch 47/50
2000/2000 [==============================] - 1s 723us/step - loss: 0.0418 - acc: 0.9970 - val_loss: 0.4497 - val_acc: 0.9675
Epoch 48/50
2000/2000 [==============================] - 1s 726us/step - loss: 0.0320 - acc: 0.9980 - val_loss: 0.4128 - val_acc: 0.9700
Epoch 49/50
 656/2000 [========>.....................

2000/2000 [==============================] - 1s 725us/step - loss: 0.0320 - acc: 0.9980 - val_loss: 0.3702 - val_acc: 0.9700
Epoch 50/50
2000/2000 [==============================] - 1s 727us/step - loss: 0.0320 - acc: 0.9980 - val_loss: 0.2510 - val_acc: 0.9725


In [22]:
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               4194560   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 4,194,817
Trainable params: 4,194,817
Non-trainable params: 0
_________________________________________________________________


In [23]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

for x_batch, y_batch in validation_generator:
  print(y_batch)
  print(model_1.predict_proba(model.predict(x_batch)))
  break


Found 400 images belonging to 2 classes.
[0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0.]
[[0.000000e+00]
 [1.000000e+00]
 [3.014343e-38]
 [0.000000e+00]
 [1.000000e+00]
 [1.000000e+00]
 [1.000000e+00]
 [0.000000e+00]
 [0.000000e+00]
 [0.000000e+00]
 [1.000000e+00]
 [1.000000e+00]
 [0.000000e+00]
 [1.000000e+00]
 [1.000000e+00]
 [4.628651e-21]]
